In [3]:
import numpy as np

# Metrics for active learning

## Contradictions

We define a first metric based on contradictions. It has been observed that the number of samples on which the model changes his prediction from one iteration to the other is correlated to the improvement of accuracy. We want to verify this. Since the number of label prediction changes can be coarse, we use the absolute difference in prediction probabilities.

In [4]:
def compute_contradiction(previous_proba, current_proba):
    return np.abs(current_proba - previous_proba).mean()

## Exploration

We define a second metric based on the distance between already labeled samples and our test set. The goal of this metric is measure how well our test set has been explored by our query sampling method so far. We expect uncertainty sampling to explore the sample space located *nearby* the decision boundary and show poor exploration property.


In [5]:
def compute_exploration(X_selected, X_test):
    return pairwise_distances(X_selected, X_test).mean()

## Compute them in an experiment

In [8]:
import openml
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder



dataset = openml.datasets.get_dataset(1483)
X, y, cat_indicator, names = dataset.get_data(dataset_format='array', target=dataset.default_target_attribute)
cat_indicator = np.asarray(cat_indicator)

ct = ColumnTransformer([
    ('encoder', OneHotEncoder(), np.where(cat_indicator)[0]),
    ('normalizer', StandardScaler(), np.where(~cat_indicator)[0])
], remainder='passthrough')

X = ct.fit_transform(X)


In [ ]:
from cardinal.uncertainty import MarginSampler
from cardinal.random import RandomSampler
from cardinal.zhdanov2019 import TwoStepKMeansSampler
from cardinal.plotting import plot_confidence_interval


samplers = [
    ('Random', RandomSampler(batch_size)),
    ('Margin', MarginSampler(model, batch_size, assume_fitted=True)),
    ('WKMeans', TwoStepKMeansSampler(10, model, batch_size, assume_fitted=True))
]

figure_accuracies = plt.figure().number
idxs = {}

for sampler_name, sampler in samplers:
    
    all_accuracies = []

    for k in range(10):
        idx = ActiveLearningSplitter.train_test_split(X.shape[0], test_size=500, random_state=k)

        accuracies = []

        # For simplicity, the first batch is simply one sample from each class
        one_per_class = np.unique(y[idx.non_selected], return_index=True)[1]
        idx.add_batch(one_per_class)

        # A classic active learning loop
        for j in range(n_iter):
            model.fit(X[idx.selected], y[idx.selected])
            accuracies.append(model.score(X[idx.test], y[idx.test]))
            sampler.fit(X[idx.selected], y[idx.selected])
            idx.add_batch(sampler.select_samples(X[idx.non_selected]))

        all_accuracies.append(accuracies)
    
    # Keep the last splitter
    idxs[sampler_name] = idx
    
    x_data = np.arange(10, batch_size * (n_iter - 1) + 11, batch_size)

    plt.figure(figure_accuracies)
    plot_confidence_interval(x_data, all_accuracies, label=sampler_name)

plt.figure(figure_accuracies)
plt.xlabel('Labeled samples')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()

plt.show()